---

# *barf*: a drop-in bioinformatics file format validator

---

## Camille Scott
### Lab for Data Intensive Biology
### March 18, 2016

---

---

## Background

---

* High-throughput DNA sequencing creates HUGE volumes of data
* Often times this data is processed through complex pipelines

---

![cost](img/costpergenome2015.jpg)

![workflow](img/workflow_galaxy.jpg)

![workflow](img/workflow.svg)

---

## Motivation

---

* Most bioinformatics software is developed by academic labs; and
* most academic labs don't have the time or money for formal verification; and
* most academic labs can't even afford software engineers; 
* AND, most users of the software are barely computationally literate

---

#### The result?



<center>
![bomb](http://bestanimations.com/Military/Explosions/nuclear-atom-bomg-explosion-animated-gif-4.gif)
</center>

---

---

### Motivation: The Story of "L"

---

"L is a new graduate student with a background in bench biology who has been diving deeper into bioinformatics as a part of her PhD research. “L” is assembling a genome, and her analysis pipeline includes the widely-used program Trimmomatic [1] to remove low-quality sequences. Some days later, when the pipeline has completed, she starts to look more closely at her results, and realizes that one of the sequence files output by Trimmomatic is truncated: the FASTQ formatted file ends part-way through a DNA sequence, and includes no quality score. This does not trigger a failure until a few steps down the pipeline, when another program mysteriously crashes. As it turns out, Trimmomatic occasionally fails due to some unpredictable error which cannot be reproduced, and instead of returning an error code, returns 0 and truncates its output. Had the program behaved more appropriately, “L” would have identified the problem early-on and saved significant time."

---

---

### Problem!

* This story is *common*
* Reporting bugs is time consuming, fixing them moreso
* Many bugs are unpredictabe or system-dependent
* Bad data gives bad results: junk in, junk out

---

*barf tries to solve this problem by allowing easy drop-in data validation for any bioinformatics program.*

---

### Aside: why the name?

Our lab likes silly names, and we discussed this concept a while back. It goes along well with my mRNA annotator, dammit :)

## Case: FASTA Format

* This barf prototype targets FASTA format
    - Widely used, poorly defined, often broken
* The expected format can be defined in BNF form as follows:

    
    <file>     ::= <token> | <token> <file>
    <token>    ::= <ignore> | <seq>
    <ignore>   ::= <whitespace> | <comment> <newline>
    <seq>      ::= <header> <molecule> <newline>
    <header>   ::= ">" <arbitrary text> <newline>
    <molecule> ::= <mol-line> | <mol-line> <molecule>
    <mol-line> ::= <nucl-line> | <prot-line>
    <nucl-line>::= "^[ACGTURYKMSWBDHVNX-]+$"
<prot-line>::= "^[ABCDEFGHIKLMNOPQRSTUVWYZX*-]+$"

### in reality....

* In reality, this format is often toyed with
* Many programs fail on the header, many mangle the sequence with line breaks, many parsers don't follow convention
* The format itself is trivial to parse; the data is what needs to be checked

---
### Approach

---

* Instead of focusing on parsing, we focus on a limited model of the data
* This is a crude type system based on regular expressions
* Can be arbitrary python code

---

In [2]:
import re
import string

class SequenceModel(object):

    def __init__(self, alphabet, flags=re.IGNORECASE):
        self.alphabet = alphabet
        self.pattern = re.compile(r'[{alphabet}]*$'.format(alphabet=alphabet),
                                  flags=flags)

    def __str__(self):
        return 'SequenceModel<{0}>'.format(self.alphabet)

    def checkValid(self, data):
        if self.pattern.match(data) is None:
            raise AssertionError('{0} failed to match "{1}"'.format(self, data))

dnaModel = SequenceModel('ACGT')
dnanModel = SequenceModel('ACGTN')
iupacModel = SequenceModel('ARNDCQEGHILKMFPSTWXVBZX')

models = {'DNA': dnaModel,
          'DNA+N': dnanModel,
          'IUPAC': iupacModel}

---

* Gives a simple framework for defined what the different *fields* in the data should look like
* The parsing is done with third-party libraries: we assume the parsers make a best-effort to consume that data
* In a way, we validate both the parser and the program

---
## What about "L"?
---

* Only validating data elements is not enough: we need to validate the data is a whole
* Introduce a collection: keep track of what inputs and outputs
* We want $OUTPUT \subseteq INPUT$ where $INPUT$ and $OUTPUT$ are sets of some record (in thise case, FASTA)

---
### Bloom Filters
---

* This data is BIG! Hundreds of millions of elements!
* Exact counting not an option
* Instead, use a bloom filter to represent the set

![bf](https://upload.wikimedia.org/wikipedia/commons/thumb/a/ac/Bloom_filter.svg/2000px-Bloom_filter.svg.png)

This way, we can assert that each element in the output is an element of the input.

---
## Implementation
---

* The invocation format is based on GNU `time`
* Pass the target program and arguments to barf; pipe input to barf; output on standard out
* barf manages the subprocess in the background: validates input, sends it to a FIFO for the program to consume

In [5]:
# a no-op
!cat test.500.fasta | ./barf --sequence-model DNA cat > test.out.fa
!head test.out.fa

FIFO constructed at /var/folders/ml/nnm5g2cn6xscbd6jd6lrqys40000gn/T/tmpThQhFs/named_pipe
FIFO destroyed at /var/folders/ml/nnm5g2cn6xscbd6jd6lrqys40000gn/T/tmpThQhFs/named_pipe
>c240946_g1_i1 len=534 path=[677:0-147 40:148-273 1221:274-533]
TTTTTTTTTTTTTTTTTCCCTTTTAAAAAATTTAATTCCAGTAAATTGTATTGCCACATGTCCACACACAAAAATACTTTTTTTGCGAAACGAACCATTGCAATGGTTTTATCAAAATTGTACAGACACAGCGAAGAGAGAGAGAGAAAACAATCGTGGAGGTCAGGCAACGCTATCTGCTGCACCAGTTGACTGAATGGAAACTAATTGAAAAGGGCCATTGTGTGTGAGATGTTTGTCTGTAATCATTCACCGCTGGGGACTCCATCAATACTCAGTGCCCAGAGCGCCCTCCCCGCAAGTGTGAAAGGTGAGATATTTCTTCTTGAGGTGTGGTTCTTGTTGCTGCTGCTAACTCTGGTGCGTGTGGCACCCAGAGGGGAAAATTTTACGTTGAATTAGACATCACAAATATAGGCCATATTGGACATAGCTAGCCTCCCCATATCTAGCTCTCTATGGCTATATAATAATTAACGTACAGCCCTTTCTCAATCCATTGACGGATGCAGGCTTCCCCATGTACTCTGCGTT
>c148030_g1_i1 len=387 path=[722:0-34 30:35-173 772:174-386]
GATTGCAAGGATCCATACTCTTTGTTTTTTTCGGGAAAGCCACGTAGGTATAAAATAATATACAAGTTACAATGTACAAGTAGTACTCATTTTTCAACCCCAAAGCGATGATAATTGACCATCTGAAGATAATTTTAACTTACAAACGTACCATTTGTGGGT

In [6]:
# a bad sequence
!cat badfasta.fa | ./barf --sequence-model DNA cat > test.out.fa

FIFO constructed at /var/folders/ml/nnm5g2cn6xscbd6jd6lrqys40000gn/T/tmpHyBIkF/named_pipe
Exception in thread Thread-1:
Traceback (most recent call last):
  File "/Users/camille/anaconda/envs/bio/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/Users/camille/anaconda/envs/bio/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "./barf", line 40, in handle_input
    record = recordModel(record.name, record.sequence)
  File "/Users/camille/w/barf/barf/fasta.py", line 15, in factory
    sequence_model.checkValid(sequence)
  File "/Users/camille/w/barf/barf/sequencemodel.py", line 17, in checkValid
    raise AssertionError('{0} failed to match "{1}"'.format(self, data))
AssertionError: SequenceModel<ACGT> failed to match "FASTALINEWITHBADTHINGS"

FIFO destroyed at /var/folders/ml/nnm5g2cn6xscbd6jd6lrqys40000gn/T/tmpHyBIkF/named_pipe


In [8]:
# we don't check biological meaning
!cat badfasta.fa | ./barf --sequence-model IUPAC cat > test.out.fa

FIFO constructed at /var/folders/ml/nnm5g2cn6xscbd6jd6lrqys40000gn/T/tmpL8bLeQ/named_pipe
FIFO destroyed at /var/folders/ml/nnm5g2cn6xscbd6jd6lrqys40000gn/T/tmpL8bLeQ/named_pipe


In [10]:
# adding in a new sequence
!cat test.500.fasta | ./barf --sequence-model DNA ./fraudster.py > /dev/null

FIFO constructed at /var/folders/ml/nnm5g2cn6xscbd6jd6lrqys40000gn/T/tmpwpxUs7/named_pipe
Exception in thread Thread-2:
Traceback (most recent call last):
  File "/Users/camille/anaconda/envs/bio/lib/python2.7/threading.py", line 801, in __bootstrap_inner
    self.run()
  File "/Users/camille/anaconda/envs/bio/lib/python2.7/threading.py", line 754, in run
    self.__target(*self.__args, **self.__kwargs)
  File "./barf", line 51, in handle_cmd
    collection.checkIn(record.sequence)
  File "/Users/camille/w/barf/barf/collectionmodel.py", line 14, in checkIn
    raise AssertionError('{0} not in input collection!'.format(data))
AssertionError: ATGGG not in input collection!

FIFO destroyed at /var/folders/ml/nnm5g2cn6xscbd6jd6lrqys40000gn/T/tmpwpxUs7/named_pipe


---
## Conclusions
---

* This is a simple prototype of a way to approach the problem
* Lab is hoping to expand this to a general tool for the community
* Needs more formats, and better performance
---